### Importing libraries

In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from merlin import LexGrouping, MeasurementStrategy, QuantumLayer
from merlin.builder import CircuitBuilder

import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

### Setting up dataset

In [ ]:
iris = load_iris()
X = iris.data.astype("float32")
y = iris.target.astype("int64")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    stratify=y,
    random_state=42,
)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Normalise features before encoding them as phases
mean = X_train.mean(dim=0, keepdim=True)
std = X_train.std(dim=0, keepdim=True).clamp_min(1e-6)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

### Defining the circuit

In [ ]:
builder = CircuitBuilder(n_modes=6)

builder.add_entangling_layer(trainable=True, name="U1")

builder.add_angle_encoding(

)

builder.add_rotations(trainable=True, name="theta")

builder.add_superpositions(depth=1, trainable=True)

### Defining the quantum layer

In [ ]:
quantum_core = QuantumLayer(
    input_size=6,
    builder=builder,
    n_photons=3,                             
    measurement_strategy=MeasurementStrategy.probs(),
)


### Defining the model

In [ ]:
model = nn.Sequential(
    quantum_core
)

### Training the model

In [ ]:
def run_experiment(model: nn.Module, epochs: int = 60, lr: float = 0.05):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        logits = model(X_train)
        loss = F.cross_entropy(logits, y_train)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        train_preds = model(X_train).argmax(dim=1)
        test_preds = model(X_test).argmax(dim=1)
        train_acc = (train_preds == y_train).float().mean().item()
        test_acc = (test_preds == y_test).float().mean().item()
    return train_acc, test_acc, train_preds, test_preds

In [ ]:
train_acc, test_acc, train_preds, test_preds = run_experiment(model, epochs=80, lr=0.05)
print(f"Train accuracy: {train_acc:.3f} – Test accuracy: {test_acc:.3f}")


In [ ]:
confusion_matrix = confusion_matrix(y_true = y_test, y_pred = test_preds)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix)

In [ ]:
disp.plot()
plt.show()